<a href="https://colab.research.google.com/github/lapaniku/generative-sentiment/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers[torch] evaluate scikit-learn --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [34]:
# import urllib.request
# urllib.request.urlretrieve("", "regen.json")

In [35]:
#prepare dataset
import datasets
from datasets import Dataset
import json

generated_tickets = json.load(open("regen.json"))

In [ ]:
generated_tickets[:1]

[{'ticket': "Just got my new coffee maker and I couldn't be happier! It's sleek, brews super fast, and the coffee tastes amazing. I love how it has a programmable timer, so I can wake up to the smell of fresh coffee. Definitely worth the purchase!",
  'input': '',
  'output': 'Strong Positive',
  'most_similar_instructions': {'I was a little disappointed since it came without a box. So, where do I leave the adapter while the card is in the camera?': 0.1971830985915493,
   "Storage prices will always go down (though I won't say at what rate over what interval of time, a la Moore's Law - I'm not that smart.) At the time I purchased it, it was a very good price for this amount of memory.  In a year's time, it will probably be a laughably expensive sum for a pittance of storage.  Oh, well.With it in my phone, I don't have to worry about the number of photos or the duration of video I take.": 0.17777777777777778,
   "im currently using it for my nokia lumia 810 and my camera, works very wel

In [36]:
tickets = [item["ticket"] for item in generated_tickets]
sentiments = [item["output"] for item in generated_tickets]
ds = Dataset.from_dict({"text": tickets, "labels": sentiments})

In [37]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/1008 [00:00<?, ? examples/s]

In [38]:
ds = tokenized_datasets.class_encode_column("labels")

Casting to class labels:   0%|          | 0/1008 [00:00<?, ? examples/s]

In [44]:
ds = ds.train_test_split(test_size=0.3, stratify_by_column="labels")

In [39]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [41]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [42]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch", report_to="none")

In [45]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=compute_metrics,
)

In [46]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.720664,0.702970
2,No log,0.499796,0.805281
3,No log,0.576686,0.775578


TrainOutput(global_step=267, training_loss=0.591040336237418, metrics={'train_runtime': 238.2919, 'train_samples_per_second': 8.876, 'train_steps_per_second': 1.12, 'total_flos': 556494871311360.0, 'train_loss': 0.591040336237418, 'epoch': 3.0})